# 해당 파일 설명
이 코드를 모두 실행하고 나면, /converted 경로에 다음의 파일이 생성됩니다.

1. plant_all_weather_feature.csv
2. plant_test_all_weather_feature.csv


## 추가 설명
위 파일은, 2_plant_preprocessing.ipynb, 3_weather_preprocessing.ipynb 두 파일에서 만들어진, 

1. plant1_by_loc_with_datetime.csv
2. plant2_by_loc_with_datetime.csv
3. plant_test_with_datetime.csv
4. ipo_dict.pkl

을 가지고 작업을 수행하여 만들어진 파일입니다. ipo_dict라는 보간 함수를 가지고, plant1, plant2, test 데이터의 시간 정보에 따른 기상 feature를 입혀줍니다. 추가적으로 일교차를 나타내는 변수도 추가합니다.



# 1. Google drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
#BASE_WORKING_DIR = "/content/gdrive/My Drive/Colab Notebooks/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 backup_0.96model
data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
lastplus_th_10.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_cond_all_plus_.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.c

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

# interpolation 함수
import scipy.interpolate as spi

# model load
import pickle
from sklearn.externals import joblib

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Get the Data

## 1. plant data 가져오기

In [ ]:
# plant 데이터 
data_plant1 = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant1_by_loc_with_datetime.csv"))
data_plant2 = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant2_by_loc_with_datetime.csv"))
data_test = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_test_with_datetime.csv"))

# unnamed: 0 제거
data_plant1.drop(['Unnamed: 0'], axis='columns', inplace=True)
data_plant2.drop(['Unnamed: 0'], axis='columns', inplace=True)
data_test.drop(['Unnamed: 0'], axis='columns', inplace=True)

# plant1, plant2 합치기
data_plant = pd.concat([data_plant1, data_plant2], ignore_index=True)

# date datetime 형식으로 변경
data_plant['date'] = data_plant['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date())
data_test['date'] = data_test['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date())

In [ ]:
data_plant

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0
...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0


In [ ]:
data_test

,mea_ddhr,plant,loc,date,time
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0
...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0


In [ ]:
data_plant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350366 entries, 0 to 350365
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   mea_ddhr     350366 non-null  object 
 1   plant        350366 non-null  float64
 2   loc          350366 non-null  float64
 3   tem_in       350366 non-null  float64
 4   hum_in       350366 non-null  float64
 5   tem_coil     350366 non-null  float64
 6   tem_out_loc  350366 non-null  float64
 7   hum_out_loc  350366 non-null  float64
 8   cond_loc     350366 non-null  float64
 9   date         350366 non-null  object 
 10  time         350366 non-null  float64
dtypes: float64(9), object(2)
memory usage: 29.4+ MB


In [ ]:
# NULL 제거 후 info
data_plant.dropna(inplace=True)
data_plant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350366 entries, 0 to 350365
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   mea_ddhr     350366 non-null  object 
 1   plant        350366 non-null  float64
 2   loc          350366 non-null  float64
 3   tem_in       350366 non-null  float64
 4   hum_in       350366 non-null  float64
 5   tem_coil     350366 non-null  float64
 6   tem_out_loc  350366 non-null  float64
 7   hum_out_loc  350366 non-null  float64
 8   cond_loc     350366 non-null  float64
 9   date         350366 non-null  object 
 10  time         350366 non-null  float64
dtypes: float64(9), object(2)
memory usage: 32.1+ MB


## 2. Weather data 가져오기

In [ ]:
# interpolation dictionary 설명서를 load 한다.
IPO_DICT_READ_ME = pd.read_csv(os.path.join(MODEL_PATH, "IPO_DICT_READ_ME.csv"))
IPO_DICT_READ_ME = IPO_DICT_READ_ME.fillna(" ")
IPO_DICT_READ_ME

,WHERE :,dangjin,sinpyeong,seosan,songsan,songak
0,TARGET :,ta,ta,ta,ta,ta
1,,hm,ws,hm,hm,hm
2,,ws,rn,ws,ws,ws
3,,rn,lp,rn,rn,rn
4,,,ap,dew,,
5,,,,sp,,
6,,,,lp,,
7,,,,ap,,
8,,,,sr,,
9,,,,ss,,


In [ ]:
# 설명서 기반으로 dictionary 만들기
weather_name = IPO_DICT_READ_ME.columns[1:]
weather_cnt = [4, 5, 11, 4, 4]
weather = dict()
idx = 0
for name in weather_name:
  weather[name] = IPO_DICT_READ_ME[name][:weather_cnt[idx]]
  idx += 1
weather.keys()

dict_keys(['dangjin', 'sinpyeong', 'seosan', 'songsan', 'songak'])

In [ ]:
# interpolation dictionary 를 load 한다.
import pickle
ipo_dict = joblib.load(os.path.join(MODEL_PATH, "ipo_dict.pkl"))
ipo_dict.keys()

dict_keys([('dangjin', 'ta', datetime.date(2016, 4, 1)), ('dangjin', 'ta', datetime.date(2016, 4, 2)), ('dangjin', 'ta', datetime.date(2016, 4, 3)), ('dangjin', 'ta', datetime.date(2016, 4, 4)), ('dangjin', 'ta', datetime.date(2016, 4, 5)), ('dangjin', 'ta', datetime.date(2016, 4, 6)), ('dangjin', 'ta', datetime.date(2016, 4, 7)), ('dangjin', 'ta', datetime.date(2016, 4, 8)), ('dangjin', 'ta', datetime.date(2016, 4, 9)), ('dangjin', 'ta', datetime.date(2016, 4, 10)), ('dangjin', 'ta', datetime.date(2016, 4, 11)), ('dangjin', 'ta', datetime.date(2016, 4, 12)), ('dangjin', 'ta', datetime.date(2016, 4, 13)), ('dangjin', 'ta', datetime.date(2016, 4, 14)), ('dangjin', 'ta', datetime.date(2016, 4, 15)), ('dangjin', 'ta', datetime.date(2016, 4, 16)), ('dangjin', 'ta', datetime.date(2016, 4, 17)), ('dangjin', 'ta', datetime.date(2016, 4, 18)), ('dangjin', 'ta', datetime.date(2016, 4, 19)), ('dangjin', 'ta', datetime.date(2016, 4, 20)), ('dangjin', 'ta', datetime.date(2016, 4, 21)), ('dangjin',

## 3. 공장 + 기상 데이터 제작하기 (Train)

In [ ]:
# data_plant에 ipo_dict를 기준으로 기상 데이터를 추가한다.
# 굉장히 오래 걸리므로 파일입출력으로 해결한다.
for i in range(len(weather_name)):
  print(weather_name[i])
  for j in range(weather_cnt[i]):
    feature = weather[weather_name[i]][j]
    data_plant[weather_name[i] + '-' + feature] = 0

  date_start = min(data_plant['date'])
  date_end = max(data_plant['date'])
  date = date_start

  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print(date)
    tmp = data_plant[data_plant['date'] == date]
    for j in range(weather_cnt[i]):
      feature = weather[weather_name[i]][j]
      key = (weather_name[i], feature, date)
      ipo = ipo_dict[key]
      tmp[weather_name[i] + '-' + feature] = tmp['time'].apply(lambda x : spi.splev(x, ipo))
    data_plant.loc[tmp[weather_name[i] + '-' + weather[weather_name[i]][0]].index] = tmp
    date += timedelta(days=1)
data_plant

dangjin
2017-01-01
2018-01-01
2019-01-01
sinpyeong
2017-01-01
2018-01-01
2019-01-01
seosan
2017-01-01
2018-01-01
2019-01-01
songsan
2017-01-01
2018-01-01
2019-01-01
songak
2017-01-01
2018-01-01
2019-01-01


,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0


## 4. 공장 + 기상 데이터 제작하기 (Test)

In [ ]:
# dictionary로 부터 기상 값 채워 넣기
for i in range(len(weather_name)):
  print(weather_name[i])
  for j in range(weather_cnt[i]):
    feature = weather[weather_name[i]][j]
    data_test[weather_name[i] + '-' + feature] = 0

  date_start = min(data_test['date'])
  date_end = max(data_test['date'])
  date = date_start

  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print(date)
    tmp = data_test[data_test['date'] == date]
    for j in range(weather_cnt[i]):
      feature = weather[weather_name[i]][j]
      key = (weather_name[i], feature, date)
      ipo = ipo_dict[key]
      tmp[weather_name[i] + '-' + feature] = tmp['time'].apply(lambda x : spi.splev(x, ipo))
    data_test.loc[tmp[weather_name[i] + '-' + weather[weather_name[i]][0]].index] = tmp
    date += timedelta(days=1)
data_test

dangjin
2020-01-01
sinpyeong
2020-01-01
seosan
2020-01-01
songsan
2020-01-01
songak
2020-01-01


,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0


## 4. 파일 저장하기

In [ ]:
# 파일 저장하기
data_plant.to_csv(os.path.join(DATA_CONV_PATH, "plant_train_all_weather_feature.csv"))

In [ ]:
# 파일 저장하기
data_test.to_csv(os.path.join(DATA_CONV_PATH, "plant_test_all_weather_feature.csv"))

In [ ]:
data_plant.columns

Index(['mea_ddhr', 'plant', 'loc', 'tem_in', 'hum_in', 'tem_coil',
       'tem_out_loc', 'hum_out_loc', 'cond_loc', 'date', 'time', 'dangjin-ta',
       'dangjin-hm', 'dangjin-ws', 'dangjin-rn', 'sinpyeong-ta',
       'sinpyeong-ws', 'sinpyeong-rn', 'sinpyeong-lp', 'sinpyeong-ap',
       'seosan-ta', 'seosan-hm', 'seosan-ws', 'seosan-rn', 'seosan-dew',
       'seosan-sp', 'seosan-lp', 'seosan-ap', 'seosan-sr', 'seosan-ss',
       'seosan-ground', 'songsan-ta', 'songsan-hm', 'songsan-ws', 'songsan-rn',
       'songak-ta', 'songak-hm', 'songak-ws', 'songak-rn'],
      dtype='object')

In [ ]:
data_test.columns

Index(['mea_ddhr', 'plant', 'loc', 'date', 'time', 'dangjin-ta', 'dangjin-hm',
       'dangjin-ws', 'dangjin-rn', 'sinpyeong-ta', 'sinpyeong-ws',
       'sinpyeong-rn', 'sinpyeong-lp', 'sinpyeong-ap', 'seosan-ta',
       'seosan-hm', 'seosan-ws', 'seosan-rn', 'seosan-dew', 'seosan-sp',
       'seosan-lp', 'seosan-ap', 'seosan-sr', 'seosan-ss', 'seosan-ground',
       'songsan-ta', 'songsan-hm', 'songsan-ws', 'songsan-rn', 'songak-ta',
       'songak-hm', 'songak-ws', 'songak-rn'],
      dtype='object')